<a href="https://colab.research.google.com/github/LuigiElo/HalfEvoStacker/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
import requests
from bs4 import BeautifulSoup
import os.path
import json

# Send a GET request to the website
baseURL = "https://www.futwiz.com"

In [159]:
import re

def convert_expiration_to_hours(expiration_str):
    # Extract the numeric value and the time unit
    match = re.match(r"Expires: (\d+) (weeks|days|hours|months)", expiration_str)

    if match:
        value, unit = int(match.group(1)), match.group(2)

        # Define conversion factors
        conversion_factors = {
            'weeks': 7 * 24,    # 1 week = 7 days
            'days': 24,         # 1 day = 24 hours
            'hours': 1,
            'months': 30 * 24   # Assuming 30 days per month
        }

        # Convert to hours
        hours = value * conversion_factors.get(unit, 1)
        return hours
    else:
        # Handle invalid input
        return None

In [160]:
def getAllActiveEvos():
  all_evos = []
  response = requests.get(f'{baseURL}/en/fc24/evolutions')
  # Parse the HTML content
  soup = BeautifulSoup(response.content, 'lxml')
  cards = soup.find_all('div', class_='evo-block')

  for card in cards:
    requires = card.find('div', class_='evo-requires')
    upgrades = card.find('div', class_='evo-upgrades')
    top = card.find('div', class_='evo-top')

    onclick_value = card.get('onclick')
    evo_path = onclick_value.split("'")[1]
    print(f'{baseURL}{evo_path}')

    res = requests.get(f'{baseURL}{evo_path}')
    su = BeautifulSoup(res.content, 'lxml')

    evo_level_divs = su.find_all('div', class_='evo-level')

    levels = []
    # Iterate over each "evo-level" div
    for evo_level_div in evo_level_divs:
        # Create an object for each level
        level_object = {
            "level_name": evo_level_div.h4.text.strip(),
            "rewards": {}
        }
        # Extract rewards
        for reward_element in evo_level_div.find_all('div', class_='evo-lv-reward'):
            reward_text = reward_element.text.strip()
            reward_key, reward_value = map(str.strip, reward_text.split(':', 1))

            # Handle PlayStyles separately
            if 'playstyles' == reward_key.lower():
                image_src = reward_element.find('img')['src']
                image_name = image_src.split('/')[-1].split('.')[0]
                level_object["rewards"]["playstyles"] = image_name
            elif 'playstyles+' == reward_key.lower():
                image_src = reward_element.find('img')['src']
                image_name = image_src.split('/')[-1].split('.')[0]
                level_object["rewards"]["playstyles+"] = image_name
            else:
                level_object["rewards"][reward_key.lower()] = reward_value


        # Add the level object to the list
        levels.append(level_object)

    evo_requires_data = {}
    evo_upgrades_data = {}

    title = top.find('h3').text.strip()
    expires = top.find('div').text.strip()
    hours = convert_expiration_to_hours(expires)

    # Iterate over each div with class "evo-list" within the evo-requires section
    for evo_list in requires.find_all('div', class_='evo-list'):
        category = evo_list.find_all('div')[0].text.strip()
        value = evo_list.find_all('div')[1].text.strip()

        if category.lower() == "excluded positions":
          evo_requires_data[category.lower()] = [pos.strip() for pos in value.split('/')]
          print([pos.strip() for pos in value.split('/')])
        else:
          evo_requires_data[category.lower()] = value

    for evo_list in upgrades.find_all('div', class_='evo-list'):
        category = evo_list.find_all('div')[0].text.strip()
        boost_value_element = evo_list.find('div', class_='evo-list-boost')

        if boost_value_element:
            boost_value = boost_value_element.text.strip()
            # If the category is "PlayStyles," use the image name from the src attribute
            if 'playstyles' in category.lower():
                boost_value = [os.path.splitext(os.path.basename(img['src']))[0] for img in boost_value_element.find_all('img')]
        else:
            boost_value = None

        # Convert category to lowercase and add it as an attribute to the dictionary
        evo_upgrades_data[category.lower()] = boost_value


    evo = { 'requires': evo_requires_data, 'upgrades': evo_upgrades_data, "title":title, "expires":hours, "levels": levels }
    all_evos.append(evo)
  return all_evos

In [161]:
def getPlayer(playerURL):
  response = requests.get(playerURL)
  soup = BeautifulSoup(response.content, 'lxml')


  position = soup.find('div', class_='card-24-position').text.strip()
  overall = soup.find('div', class_='card-24-rating').text.strip()
  player_info = soup.find('div', class_='player-details-inner')

  # Find all player-detail-row divs
  player_detail_rows = player_info.find_all('div', class_='player-detail-row')

  # Create a dictionary to store player details
  player_details = {}

  # Iterate over each player-detail-row div
  for detail_row in player_detail_rows:
      # Extract detail name and value
      detail_name = detail_row.find('div').text.strip()
      detail_value = detail_row.find_all('div')[1].text.strip()

      # Handle Alt Pos. separately
      if detail_name == 'Alt Pos.':
          alt_positions = [pos.text for pos in detail_row.find_all('span')]
          player_details['alt positions'] = alt_positions
      else:
          player_details[detail_name] = detail_value

  player_details['position']= position
  player_details['overall']= overall

  # Find all stats-grid-block divs
  stats_grid_blocks = soup.find_all('div', class_='stats-grid-block')

  # Create a list to store stats
  player_stats = {}

  # Iterate over each stats-grid-block div
  for stats_block in stats_grid_blocks:
      # Extract headline stats
      headline_stat_label = stats_block.find('div', class_='headline-stat-label').text.strip()
      headline_stat_num = stats_block.find('div', class_='headline-stat-num').text.strip()
      player_stats[headline_stat_label] = headline_stat_num

      # Extract individual stats
      individual_stat_containers = stats_block.find_all('div', class_='individual-stat-container')
      for stat_container in individual_stat_containers:
          stat_label = stat_container.find('div', class_='individual-stat-bar-label').text.strip()
          stat_value = stat_container.find('div', class_='individual-stat-bar-stat').text.strip()
          player_stats[stat_label] = stat_value

  playstyles_plus_heading = soup.find('h3', text='PlayStyles+')
  playstyles_heading = soup.find('h3', text='PlayStyles')

  playstyles = 0
  playstyles_plus = 0

  # Check if PlayStyles+ section exists
  if playstyles_plus_heading:
      playstyles_plus_block = playstyles_plus_heading.find_next('div', class_='player-playstyle-block')
      playstyles_plus_div = playstyles_plus_heading.find_next('div')
      if playstyles_plus_block and 'no PlayStyles+' not in playstyles_plus_div.text:
          playstyles_plus = len(playstyles_plus_block.find_all('div', class_='player-playstyle-info'))

  # Check if PlayStyles section exists
  if playstyles_heading:
      playstyles_block = playstyles_heading.find_next('div', class_='player-playstyle-block')
      playstyles_div = playstyles_heading.find_next('div')
      if playstyles_block and 'no PlayStyles' not in playstyles_div.text:
          playstyles = len(playstyles_block.find_all('div', class_='player-playstyle-info'))


  return {"info": player_details,  "stats":player_stats, "playstyles+":playstyles_plus, "playstyles" : playstyles}


In [176]:
def check_evolution(player, evo):
    attribute_mapping = {
        'pace': 'pac',
        'shooting': 'sho',
        'passing': 'pas',
        'defending': 'def',
        'physical': 'phy',
        'dribbling': 'dri',
        # Add more mappings as needed
    }
    evolution_requirements = evo.get('requires', {})

    # Merge 'info' and 'stats' into 'player' if both keys exist
    if 'info' in player and 'stats' in player:
        player.update(player['info'])
        player.update(player['stats'])

    # Convert all player attributes to lowercase
    player_lower = {k.lower(): v for k, v in player.items()}

    positions = [player_lower.get('position')]
    alt_positions = player_lower.get('alt positions', '')

    required_position = evolution_requirements.pop('position', None)
    excluded_positions = evolution_requirements.pop('excluded positions', [])
    max_alt_positions_str = evolution_requirements.pop('alt positions', None)
    max_alt_positions = int(max_alt_positions_str.split()[-1]) if max_alt_positions_str and 'max' in max_alt_positions_str.lower() else None

    # Check max alt_positions count
    if max_alt_positions is not None and len(alt_positions) > max_alt_positions:
        print(f"Failed: More than {max_alt_positions} alt positions are not allowed")
        return False

    # Check excluded positions in positions and alt_positions
    excluded_positions_lower = [pos.lower().strip() for pos in excluded_positions]
    if any(pos in excluded_positions_lower for pos in (pos.lower() for pos in positions + alt_positions)):
        print(f"Failed: Excluded position found in positions or alt_positions")
        return False

    # Check required position in positions and alt_positions
    if required_position is not None and required_position.lower() not in (pos.lower() for pos in positions + alt_positions):
        print(f"Failed: Required position not found in positions or alt_positions")
        return False

    for attr, value in evolution_requirements.items():
        # Convert attribute names to lowercase for case-insensitive comparison
        attr_lower = attribute_mapping.get(attr.lower(), attr.lower())

        if attr_lower in player_lower:
            player_value = int(player_lower[attr_lower]) if isinstance(player_lower[attr_lower], str) and player_lower[attr_lower].isdigit() else player_lower[attr_lower]

            if isinstance(value, str) and 'max' in value.lower():
                max_value = int(value.lower().split()[-1])
                if player_value > max_value:
                    print(f"Failed: {attr_lower} > {max_value}")
                    return False
            elif isinstance(value, str) and 'min' in value.lower():
                min_value = int(value.lower().split()[-1])
                if player_value < min_value:
                    print(f"Failed: {attr_lower} < {min_value}")
                    return False
            elif isinstance(value, str) and 'max' not in value.lower() and 'min' not in value.lower():
                if player_value.lower() != value.lower():
                    print(f"Failed: {attr_lower} != {value.lower()}")
                    return False
            elif isinstance(value, int):
                if player_value != value:
                    print(f"Failed: {attr_lower} != {value}")
                    return False
            else:
                return False
        else:
            print(f"Failed: {attr_lower} not in player attributes")
            return False

    return True



def evolutionsThatFitPlayer(player,evolutions):
  fitting_evos = []
  for evolution in evolutions:
      copied_evolution = json.loads(json.dumps(evolution))
      copied_player_data = json.loads(json.dumps(player))

      upgraded_player = check_evolution(copied_player_data, copied_evolution)
      if upgraded_player:
        print(f'-------------------{copied_evolution["title"]}---------------------------')
        fitting_evos.append(evolution)
  return fitting_evos

def evolutionsThatFitEvos(level,evolutions):
  fitting_evos = []
  for evolution in evolutions:
      copied_evolution = json.loads(json.dumps(evolution))
      copied_level = json.loads(json.dumps(level))

      upgraded_player = check_evolution(copied_level["Player"], copied_evolution)

      eligible = True
      if copied_level["Status"] == "Active":
        eligible = False
      elif copied_level["Status"] == "Expired" and copied_evolution["expires"] <= copied_level["Expires"]:
        eligible = False

      if upgraded_player and eligible:
        print(f'-------------------{copied_evolution["title"]}---------------------------')
        fitting_evos.append(evolution)
  return fitting_evos

def apply_attribute_upgrade(player, attribute, upgrade_value,section="stats"):
    #print(f'{player} {attribute} {upgrade_value}')
    current_value = int(player[section][attribute])
    sign = upgrade_value[0]  # Get the sign (+ or -)
    value = int(upgrade_value[1:])  # Get the numerical value
    result = current_value

    if sign == '+':
        result = current_value + value
    elif sign == '-':
        result = current_value - value
    else:
        result = current_value  # No sign, treat it as an addition

    return result

def upgradeEvolutionLevel(player, evolution):
  attribute_mapping = {
        'pace': ['PAC', 'Acceleration', 'Sprint Speed'],
        'shooting': ['SHO', 'Positioning', 'Finishing', 'Shot Power', 'Long Shots', 'Volleys', 'Penalties'],
        'passing': ['PAS', 'Vision', 'Crossing', 'FK. Acc.', 'Short Pass', 'Long Pass', 'Curve'],
        'dribbling': ['DRI', 'Agility', 'Balance', 'Reactions', 'Ball Control', 'Dribbling', 'Composure'],
        'defending': ['DEF', 'Interceptions', 'Heading Acc.', 'Def. Awareness', 'Stand Tackle', 'Slide Tackle'],
        'physical': ['PHY', 'Jumping', 'Stamina', 'Strength', 'Aggression'],
        'overall': ["overall"],
        'weak foot': ["Weak Foot"],
        'skill moves': ["Skill Moves"],
        'new positions': ["alt positions"],
        'playstyles': ["playstyles"],
        'playstyles+': ["playstyles+"],
        'short passing': ['Short Pass'],
        'long passing': ['Long Pass']
    }

  copied_evolution = json.loads(json.dumps(evolution))
  copied_player_data = json.loads(json.dumps(player))

  player_levels = []

  for level in copied_evolution["levels"]:
    for category, attributes in attribute_mapping.items():
        if category in level["rewards"]:
          for stat in attributes:
            if stat in ["overall", "Weak Foot", "Skill Moves"]:
              copied_player_data["info"][stat] = apply_attribute_upgrade(copied_player_data,stat,level["rewards"][category],"info")
            elif stat in ["alt positions"]:
              copied_player_data["info"][stat].append(level["rewards"][category][1:])
            elif "playstyles" in stat:
              copied_player_data[stat] += 1
            else:
              copied_player_data["stats"][stat] = apply_attribute_upgrade(copied_player_data,stat,level["rewards"][category])
          level["rewards"].pop(category)
    for up in level["rewards"]:
        copied_player_data["stats"][up.title()] = apply_attribute_upgrade(copied_player_data,up.title(),level["rewards"][up])



    stack = copied_player_data.get('stack',[])
    stack.append(f'{evolution["title"]} - {level["level_name"]}')
    copied_player_data["stack"] = stack

    level_copy =  { "Evolution": f'{evolution["title"]} - {level["level_name"]}', "Player": json.loads(json.dumps(copied_player_data)), "Status": "Active", "Expires": evolution["expires"] }
    player_levels.append(level_copy)

  player_levels[-1]["Status"] = "Complete"
  player_levels[-2]["Status"] = "Expired"
  return player_levels


def findStacks(player,evolutions):
  if len(evolutions) == 0:
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Final Evo>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

  all_levels = [upgradeEvolutionLevel(player, fitting_evo) for fitting_evo in evolutions]
  evo_path = []
  for levels in all_levels:
    for level in levels:
      if level["Status"] is not "Active":
        print(f':::::::::::::::::{level["Evolution"]}:::::::::::::::::::::')
        print(f':::::::::::::::::{level["Player"]["info"]["overall"]}:::::::::::::::::::::')
        print(f':::::::::::::::::{level["Player"]["stack"]}:::::::::::::::::::::')
        print(f'::::::::::::::::: PAC {level["Player"]["stats"]["PAC"]} SHO {level["Player"]["stats"]["SHO"]} PAS {level["Player"]["stats"]["PAS"]}:::::::::::::::::::::')
        print(f'::::::::::::::::: DRI {level["Player"]["stats"]["DRI"]} DEF {level["Player"]["stats"]["DEF"]} PAS {level["Player"]["stats"]["PHY"]}:::::::::::::::::::::')

      print("**************************Possible Stacks****************************")
      fitting = evolutionsThatFitEvos(level, [evo for evo in evolutions if evo["title"] not in level["Evolution"]])
      print("*********************************************************************")
      up_player = level['Player']
      if len(fitting) > 0:
        name, path = findStacks(up_player, fitting)
        evo_path.append(f'-> {name["Evolution"]} {name["Player"]["info"]["overall"]} ')
        print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<Final Evo>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
      else:
        continue
  return all_levels[-1][-1], evo_path

def checkPlayerStacks(player, evos):
  copied_evolution = json.loads(json.dumps(evos))
  copied_player_data = json.loads(json.dumps(player))

  print("**************************Possible Evos****************************")
  fitting_evos = evolutionsThatFitPlayer(copied_player_data,copied_evolution)
  print("******************************************************************")
  findStacks(copied_player_data,fitting_evos)


<>:188: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:188: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-176-d317e519fe7d>:188: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if level["Status"] is not "Active":


In [ ]:
evos = getAllActiveEvos()

In [ ]:
def getAllPlayersFromSearchPage(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'lxml')

  # Find all <a> tags with class "latest-player-card"
  player_cards = soup.find_all('a', class_='latest-player-card')

  # Extract href attributes and store them in a list
  players = [getPlayer(f'{baseURL}{player["href"]}') for player in player_cards]

  return players

players = getAllPlayersFromSearchPage("https://www.futwiz.com/en/fc24/players?page=0&leagues[]=2228&sex=women")
players[0]

In [ ]:
player_url= "https://www.futwiz.com/en/fc24/player/dominic-calvert-lewin/1283"
player = getPlayer(player_url)

checkPlayerStacks(player, evos)

In [177]:
checkPlayerStacks(players[2], evos)

**************************Possible Evos****************************
Failed: Required position not found in positions or alt_positions
Failed: Required position not found in positions or alt_positions
-------------------Patrick Who---------------------------
-------------------Keep Up---------------------------
Failed: Required position not found in positions or alt_positions
Failed: Required position not found in positions or alt_positions
Failed: Required position not found in positions or alt_positions
Failed: Required position not found in positions or alt_positions
Failed: Required position not found in positions or alt_positions
Failed: Required position not found in positions or alt_positions
Failed: overall > 75
Failed: Excluded position found in positions or alt_positions
-------------------Like a Rock---------------------------
Failed: rarity not in player attributes
Failed: rarity not in player attributes
Failed: phy > 69
Failed: phy > 69
Failed: Required position not found i